# Generate our Core Dataset
To be used as the base for all models.  All subsequent datasets should just be built off of this. Creates a single flat dataset. Where datasets can be generated by separating by node values then slicing the right timesteps. Generates the following dataset:
  * Time, node, hour, day, month, year, season, solar values, wind values, and load demand (y variable)

For now this only includes training data as validation may take some more time (we have to deal with forecasts at each timestep). 

In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import gc
gc.collect()

0

In [2]:
raw_dir = "../data/RE-Europe/"
processed_dir = "../data/processed/"

In [3]:
# for testing
df = pd.read_csv(processed_dir + "holidays and seasons.csv")
df.head()

,Time,dow,month,year,hour,season,1,2,3,4,...,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514
0,2012-01-01 00:00:00,6,1,2012,0,winter,1,1,1,1,...,1,1,1,1,1,1,1,0,1,1
1,2012-01-01 01:00:00,6,1,2012,1,winter,1,1,1,1,...,1,1,1,1,1,1,1,0,1,1
2,2012-01-01 02:00:00,6,1,2012,2,winter,1,1,1,1,...,1,1,1,1,1,1,1,0,1,1
3,2012-01-01 03:00:00,6,1,2012,3,winter,1,1,1,1,...,1,1,1,1,1,1,1,0,1,1
4,2012-01-01 04:00:00,6,1,2012,4,winter,1,1,1,1,...,1,1,1,1,1,1,1,0,1,1


In [4]:
# we need to melt this
def melt_df(df, id_vars, melted_vars):
    melted_df = pd.melt(df, id_vars=id_vars)
    id_vars += melted_vars
    melted_df.columns = id_vars
    
    return melted_df


In [5]:
melted_df = melt_df(df, ['Time', 'dow', 'month', 'year', 'hour', 'season'], ['node', 'holiday'])
melted_df

,Time,dow,month,year,hour,season,node,holiday
0,2012-01-01 00:00:00,6,1,2012,0,winter,1,1
1,2012-01-01 01:00:00,6,1,2012,1,winter,1,1
2,2012-01-01 02:00:00,6,1,2012,2,winter,1,1
3,2012-01-01 03:00:00,6,1,2012,3,winter,1,1
4,2012-01-01 04:00:00,6,1,2012,4,winter,1,1
...,...,...,...,...,...,...,...,...
39298171,2014-12-31 19:00:00,2,12,2014,19,winter,1514,0
39298172,2014-12-31 20:00:00,2,12,2014,20,winter,1514,0
39298173,2014-12-31 21:00:00,2,12,2014,21,winter,1514,0
39298174,2014-12-31 22:00:00,2,12,2014,22,winter,1514,0


In [ ]:
# let's do this to all of our datasets
# I am just going to manually define the loop since it is quite quick. I am also inclduing both sets of wind / solar values
# we will just need to decide which one to use later on
loop = [(raw_dir + "Nodal_TS/load_signal.csv", ["Time"], ["node", "load"]),
        (raw_dir + "Nodal_TS/solar_signal_COSMO.csv", ["Time"], ["node", "solar_cosmo"]),
        (raw_dir + "Nodal_TS/solar_signal_ECMWF.csv", ["Time"], ["node", "solar_ecmwf"]),
        (raw_dir + "Nodal_TS/wind_signal_COSMO.csv", ["Time"], ["node", "wind_cosmo"]),
        (raw_dir + "Nodal_TS/wind_signal_ECMWF.csv", ["Time"], ["node", "wind_ecmwf"]),
        (processed_dir + "holidays and seasons.csv", ['Time', 'dow', 'month', 'year', 'hour', 'season'], ['node', 'holiday'])]

In [ ]:
df = pd.read_csv(loop[0][0])
df.head()

,Time,1,2,3,4,5,6,7,8,9,...,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514
0,2012-01-01 00:00:00,75.6549,557.8868,15.5237,8.6425,85.3649,1215.1364,144.1203,21.7397,56.5293,...,47.4590,129.3250,106.4375,80.4775,0.6411,96.4397,1213.4104,7.4034,94.9570,61.1324
1,2012-01-01 01:00:00,70.9958,535.9099,14.9262,8.1865,82.0792,1168.3658,138.5731,20.9030,54.3535,...,45.0893,124.5916,101.3186,77.8000,0.6176,93.2193,1174.0210,7.0248,93.5152,58.8188
2,2012-01-01 02:00:00,66.6382,510.6472,14.2310,7.7310,78.2565,1113.9500,132.1192,19.9294,51.8220,...,42.0898,118.0562,95.5555,72.9376,0.5852,89.5650,1128.7963,6.7558,87.7891,55.1939
3,2012-01-01 03:00:00,62.9920,471.8010,13.1365,7.2408,72.2379,1028.2788,121.9582,18.3967,47.8365,...,40.1332,110.9497,89.8478,68.5877,0.5500,85.6357,1078.4654,6.5747,82.6677,51.9912
4,2012-01-01 04:00:00,61.0699,441.8839,12.2862,6.9244,67.5621,961.7206,114.0641,17.2059,44.7402,...,39.3756,107.1300,86.6700,65.2452,0.5311,83.1339,1046.3704,6.5429,79.2732,50.1999


In [ ]:
dfs = []

for val in loop:
    df = pd.read_csv(val[0])
    
    # melt and append the dataframe
    melted_df = melt_df(df, val[1], val[2])
    display(melted_df.head())
    dfs.append(melted_df)

,Time,node,load
0,2012-01-01 00:00:00,1,75.6549
1,2012-01-01 01:00:00,1,70.9958
2,2012-01-01 02:00:00,1,66.6382
3,2012-01-01 03:00:00,1,62.9920
4,2012-01-01 04:00:00,1,61.0699


,Time,node,solar_cosmo
0,2012-01-01 00:00:00,1,0.0
1,2012-01-01 01:00:00,1,0.0
2,2012-01-01 02:00:00,1,0.0
3,2012-01-01 03:00:00,1,0.0
4,2012-01-01 04:00:00,1,0.0


,Time,node,solar_ecmwf
0,2012-01-01 00:00:00,1,0.0
1,2012-01-01 01:00:00,1,0.0
2,2012-01-01 02:00:00,1,0.0
3,2012-01-01 03:00:00,1,0.0
4,2012-01-01 04:00:00,1,0.0


,Time,node,wind_cosmo
0,2012-01-01 00:00:00,1,0.0142
1,2012-01-01 01:00:00,1,0.0167
2,2012-01-01 02:00:00,1,0.0139
3,2012-01-01 03:00:00,1,0.0195
4,2012-01-01 04:00:00,1,0.0197


,Time,node,wind_ecmwf
0,2012-01-01 00:00:00,1,0.0284
1,2012-01-01 01:00:00,1,0.0336
2,2012-01-01 02:00:00,1,0.0392
3,2012-01-01 03:00:00,1,0.0424
4,2012-01-01 04:00:00,1,0.0475


,Time,dow,month,year,hour,season,node,holiday
0,2012-01-01 00:00:00,6,1,2012,0,winter,1,1
1,2012-01-01 01:00:00,6,1,2012,1,winter,1,1
2,2012-01-01 02:00:00,6,1,2012,2,winter,1,1
3,2012-01-01 03:00:00,6,1,2012,3,winter,1,1
4,2012-01-01 04:00:00,6,1,2012,4,winter,1,1


In [ ]:
# combine and merge all of datasets
core_df = reduce(lambda  left,right: pd.merge(left,right,on=['Time', 'node'],
                                            how='left'), dfs)
core_df.head()

In [ ]:
# We also wand the country information
nodes = pd.read_csv(raw_dir + "Static_data/network_nodes.csv")
nodes['ID'] = nodes['ID'].astype(str)
core_df2 = pd.merge(core_df, nodes[['ID', 'country', 'voltage']],
                  left_on = 'node', right_on = 'ID', how = 'left')
core_df2.head()

In [ ]:
# restructure and rename
core_df2.rename(columns={'Time':'time'}, inplace=True)
final_df = core_df2[['time', 'node', 'hour', 'dow', 'month', 'year', 'holiday', 'season', 'country',
                     'voltage', 'solar_cosmo', 'solar_ecmwf', 'wind_cosmo', 'wind_ecmwf', 'load']]
final_df.head()

In [ ]:
# first let's save the mins and maxes so we can back out the original values
node_min_max = final_df.groupby("node")['load'].agg(min_load = min, max_load = max).reset_index()
node_min_max['node'] = node_min_max['node'].astype(int) 
node_min_max = node_min_max.sort_values(by = "node").reset_index(drop = True)

node_min_max.to_csv(processed_dir + "Load Min Max Values.csv", index = False)

In [ ]:
del nodes, dfs, core_df, core_df2, df, melted_df, node_min_max
gc.collect()

In [ ]:
# scale each node's load to be between 0 and 1 using min max scaling
loads = []
nodes_grouped = final_df.groupby("node")
for name, group in nodes_grouped:
    min_load = min(group['load'])
    max_load = max(group['load'])
    norm_load = (group['load'] - min_load)/(max_load - min_load)
    norm_load = pd.DataFrame(norm_load, columns = ["load"])
    norm_load['node'] = name
    norm_load['time'] = group['time']
    loads.append(norm_load)

# merge back to original
norm_load = pd.concat(loads, axis = 0)

final_df2 = pd.merge(final_df.drop(columns = "load"), norm_load, on = ["node", 'time'], how = "left")
final_df2.head()

In [ ]:
# some load values are all zeros. We need to remove these
nodes2rm = ["26", "28","216","698", "1250", "1251", "1335"]
final_df2 = final_df2.loc[~final_df2['node'].isin(nodes2rm)]

In [ ]:
final_df2.load.isnull().sum()

In [ ]:
final_df2.head()

In [18]:
final_df2.to_parquet(processed_dir + "EnergyDemandData.parquet")

C:\Users\ABell\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_dtype(pandas_collection)


In [ ]:
list(final_df.columns)

In [ ]:
# only keep longitudinal columns
cols2keep = ["time", "node", "dow", "season", "holiday", "load"]
final_df_sub = final_df2[cols2keep]
display(final_df)

In [19]:
partial_df = final_df2[final_df2['node'].astype(int) < 20]
partial_df.to_parquet(processed_dir + "EnergyDemandData_Partial.parquet")